In [1]:
# preprocessing with dask
import os, sys, re, io, pathlib
import pandas as pd
import hiplot as hip
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import itertools

buffer = io.StringIO()
idx = pd.IndexSlice

# define the current path (notebooks in lab_utils)
labutilspath = str(pathlib.Path(os.getcwd()).parents[1])
sys.path.append(labutilspath)

# import the autoscan routines
from autoscan import autoscan

pp = autoscan.basics(material_info = True)

def pprint(msg, msg_title = '', msg_decorator = '#', len_decorator = 40):
    nhead = len_decorator - len(msg_title) - 2
    if nhead <= 0:
        nhead = 1
        nfoot = len(msg_title) + 4
    else:
        nfoot = len_decorator
    
    top_decorator = msg_decorator * (nhead // 2) 
    print(top_decorator + ' ' + msg_title  +  ' ' + top_decorator, 
          msg, nfoot * '#' + '\n',
          sep = '\n')
    return

def dfinfo(df, header = 'info'):
    with io.StringIO() as buffer:
        df.info(buf = buffer)
        pprint(buffer.getvalue(), msg_title = header)
    return

def idx_peak_to_lambda(x):
    if np.logical_and(x != np.nan, type(x) == str):
        out = pp.ftir_lambdas[int(x.split('_')[1]) - 1]
    else:
        out = np.nan
    return out

def ftir_row_stats(df):
    return (
        df
        .assign(
            l_mean = lambda df: df.loc[:, pp.ftir_cols].mean(axis = 1),          
            l_std = lambda df: df.loc[:, pp.ftir_cols].std(axis = 1),
            # l_median = lambda df: np.median(df.iloc[:, 2:1754], axis = 1)
        )
    )

def ftir_extreme_locations(df):
    idx_max_peaks = df.loc[:, pp.ftir_cols].idxmax(axis = 1)
    idx_min_peaks = df.loc[:, pp.ftir_cols].idxmin(axis = 1)
    return (
        df
        .assign(
            l_max_peak = idx_max_peaks.apply(lambda x: idx_peak_to_lambda(x)),
            l_min_peak = idx_min_peaks.apply(lambda x: idx_peak_to_lambda(x))        
        )
    
    )
    
def clean_dataframe(df):
    return (
        df
        .where(df >= 0, np.nan)
        .astype(np.float32)
    )

def enforce_limits(df):
    for k, p in pp.probe_settings.items():
        v = p['col'][2:]
        vmin, vmax = p['limits']
        df[v] = df[v].where(((df[v] >= vmin) & (df[v] <= vmax)), np.nan)
    return df

In [2]:
# define paths
datapath = '/sandbox/data/autoscan/'
savepath = datapath

datafile = os.path.join(datapath, 'autoscan.h5')

In [3]:
# load the data
df = pd.read_hdf(datafile, key = 'data')
df_description = pd.read_hdf(datafile, key = 'description')

In [4]:
# fix some tags
df.loc[df.tag.str.contains('eur'), 'family'] = 'shale'
df.loc[df.tag.str.contains('eur'), 'code'] = 'sh'

In [5]:
ds = df.loc[:, ['tag']]
ds = ds.assign(rho = 0.0)
ds = ds.assign(
    basetag = ds.tag.str.split('_', expand = True, n = 1)[0].values
)

for t in ds.basetag.unique():
    ds['rho'] = ds['rho'].mask(ds['basetag'] == t, pp.get_material_density(t))

ds.index.name = 'ix'
# ds = ds.set_index(['code', ds.index])

In [6]:
df.loc[:, pp.probe_cols[:8]].describe()

,perm,vp0,vs0,vp90,vs90,e_star,l_1,l_2
count,4.699700e+04,50102.000000,50100.000000,3.192300e+04,3.191600e+04,25704.000000,60630.000000,69390.000000
mean,1.109663e+03,3927.085268,2489.373980,4.981391e+03,1.508196e+04,19.090511,2.074399,1.877902
std,4.578458e+04,1085.988849,1490.192308,3.607287e+04,2.196547e+05,12.254276,0.700552,0.978563
min,4.698540e-01,864.893361,-74126.367808,-5.993123e+05,-3.181818e+06,0.486411,-0.579371,-0.632103
25%,1.850250e+00,3281.437315,1961.592486,2.886191e+03,1.964772e+03,12.255800,1.946398,1.610405
50%,3.321170e+00,3669.949232,2192.507367,3.572735e+03,2.343129e+03,15.191000,2.219440,2.263375
75%,1.176990e+02,4716.213106,2747.059583,4.666148e+03,2.674188e+03,22.335100,2.389957,2.441860
max,5.307500e+06,32211.059130,7216.618842,3.000000e+06,3.500000e+07,119.882000,6.000000,6.000000


In [7]:
# enforce limits, set non-physical values to nan
df = enforce_limits(df)

In [8]:
df.loc[:, pp.probe_cols[:8]].describe()

,perm,vp0,vs0,vp90,vs90,e_star,l_1,l_2
count,46694.000000,50085.000000,50080.000000,29821.000000,29795.000000,25704.000000,59576.000000,65637.000000
mean,114.889153,3918.563138,2510.993715,3743.132709,2401.662845,19.090511,2.115517,1.998552
std,420.262936,972.573909,832.277908,986.052753,784.020013,12.254276,0.633997,0.861626
min,0.469854,864.893361,905.689672,1283.089190,1042.768401,0.486411,0.000819,0.000177
25%,1.848690,3281.437315,1962.623702,2844.025515,1938.870196,12.255800,1.985465,1.798300
50%,3.242360,3669.724771,2192.696233,3441.889433,2232.392008,15.191000,2.224135,2.282040
75%,114.725500,4715.921850,2747.297673,4601.630292,2646.202699,22.335100,2.393898,2.454140
max,9997.370000,6837.161601,7216.618842,7907.668556,7275.120819,119.882000,6.000000,6.000000


In [18]:
# calculate the mechanical properties
# vcols = ['vp0', 'vs0', 'vp90', 'vs90']
vels  = df.loc[:, pp.vel_cols].values
vels2 = np.power(vels, 2)
rho = ds.loc[:, 'rho'].values.reshape(ds.shape[0], 1)

E = np.multiply(rho, np.multiply(vels2[:, 1::2], 3.0 * vels2[:, 0::2] - 4.0 * vels2[:, 1::2]))
df.loc[:, ['mech_e0', 'mech_e90']] = np.divide(E, vels2[:, 0::2] - vels2[:, 1::2]) / 1e6
df.loc[:, ['mech_l0', 'mech_l90']] = np.multiply(rho, vels2[:, 0::2] - 2.0 * vels2[:, 1::2])
df.loc[:, ['mech_k0', 'mech_k90']] = np.multiply(rho, vels2[:, 0::2] - (4 / 3) * vels2[:, 1::2]) / 1e6
df.loc[:, ['mech_n0', 'mech_n90']] = np.divide(vels2[:, 0::2] - 2.0 * vels2[:, 1::2], 2.* (vels2[:, 0::2] - vels2[:, 1::2]))
df.loc[:, ['mech_i0', 'mech_i90']] = np.multiply(rho, vels[:, 0::2])
df.loc[:, ['mech_m0', 'mech_m90']] = np.multiply(rho, vels2[:, 0::2])
df.loc[:, ['mech_g0', 'mech_g90']] = np.multiply(rho, vels2[:, 1::2])
df.loc[:, 'rho'] = rho

# remove the data we don't need
del E, rho, vels2

# check which values do not make sense. -1 <= pr <= 0.5
for i in pp.vel_angles:
    i = str(i)
    ix = np.logical_and(df.loc[:, 'mech_n'+i] >= -1, df.loc[:, 'mech_n'+i] <= 1.0)
    temp_cols = df.columns[df.columns.str.contains(r'mech[_][a-z]'+i)]
    df.loc[ix == False, temp_cols] = np.nan
    df.loc[ix == False, pp.meta_cols[:-1]].drop_duplicates().merge(
        df_description.loc[df_description.probe == 'vels'], 
        on = pp.meta_cols[:-1]).to_csv(os.path.join(savepath, 'repeat_vels_' + i + '.csv'))    

In [19]:
# mechs = df.columns[df.columns.str.contains('mech')]
df.loc[:, pp.mech_cols_extra].describe()

,mech_e0,mech_e90,mech_k0,mech_k90,mech_n0,mech_n90,mech_l0,mech_l90,mech_m0,mech_m90,mech_g0,mech_g90,mech_i0,mech_i90
count,46709.000000,28455.000000,46709.000000,28455.000000,46709.000000,28455.000000,4.670900e+04,2.845500e+04,4.670900e+04,2.845500e+04,4.670900e+04,2.845500e+04,46709.000000,28455.000000
mean,31.082006,29.950894,19.078286,18.127796,0.145810,0.134693,9.662829e+06,9.349978e+06,3.790920e+07,3.568343e+07,1.412319e+07,1.316673e+07,9090.195407,8769.294301
std,15.662125,15.477464,13.222407,13.633285,0.201913,0.187713,1.250199e+07,1.129654e+07,1.982006e+07,2.057639e+07,8.237217e+06,6.736441e+06,2648.301872,2883.341321
min,0.026240,0.002935,0.002918,0.000326,-0.998980,-0.999867,-3.328222e+07,-3.347834e+07,5.752067e+06,5.817813e+06,1.755386e+06,2.326963e+06,3584.546727,3570.548930
25%,18.358606,16.817258,8.560799,6.044743,0.083985,0.064560,1.332818e+06,1.077324e+06,2.351614e+07,1.775558e+07,8.285991e+06,7.839541e+06,7180.978556,6172.080743
50%,25.276381,23.923735,16.875625,14.677298,0.229270,0.201600,1.024190e+07,7.429469e+06,3.128853e+07,2.542460e+07,1.045453e+07,1.085215e+07,8550.879062,7378.462246
75%,42.646366,42.111879,26.766186,28.531940,0.267851,0.263877,1.634350e+07,1.744808e+07,5.227386e+07,5.132276e+07,1.711509e+07,1.681184e+07,11205.927102,11150.177444
max,104.161141,93.573794,88.823302,83.865764,0.444868,0.436251,7.246414e+07,6.960699e+07,1.215416e+08,1.179546e+08,5.751869e+07,6.034633e+07,18504.984887,18146.160198


In [20]:
# find the peaks of the ftir
df = ftir_extreme_locations(df)

# data cleaning
1. pre-clean the dataset
 - remove duplicated rows
 - enforce correct dtypes 
 - reduce memory overhead
 - do not remove missing values

In [ ]:
# for the record print the information of the original dataframe
dfinfo(df, 'raw data')

In [ ]:
# import klib
# # pre-clean, do not remove missing values
# df = klib.data_cleaning(df, drop_threshold_rows = 1.0, clean_col_names = False)
# df.loc[:, pp.meta_cols] = df.loc[:, pp.meta_cols].astype('object')

In [ ]:
# downcast to float32 to save some memory. this is likely useless in the current context. 
# df = df.apply(pd.to_numeric, downcast = 'float', errors = 'ignore')
# df.loc[:, pp.meta_cols] = df.loc[:, pp.meta_cols].astype('category')
# print the information of the cleaned dataframe
# dfinfo(df, 'raw data cleaned')

## fix values and correct information
1. set nan to measurements where all values are the same (ftir)
2. set the correct family and code for eur samples

In [21]:
ix = df.loc[:, pp.ftir_cols].apply(lambda x: len(np.unique(x)), axis = 1) == 1
df.loc[ix, pp.ftir_cols] = np.nan

In [35]:
col_numerical = [c for c in df.columns if c not in pp.meta_cols]

In [36]:
df.loc[:, col_numerical].to_hdf(os.path.join(savepath, 'autoscan_corrected.h5'), key = 'data', format = 'table', mode = 'w')
df.loc[:, pp.meta_cols].to_hdf(os.path.join(savepath, 'autoscan_corrected.h5'), key = 'desc', mode = 'a')

In [37]:
repeat_ftir = df.loc[ix, :].set_index(pp.meta_cols[:-1]).index.unique()
pd.DataFrame.from_records(repeat_ftir.to_numpy(), columns = pp.meta_cols[:-1]).to_csv(os.path.join(savepath, 'ftir_repeat.csv'))